### 1)-Importing Packages

In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
//import org.apache.spark.ml.clustering.{KMeans, KMeansModel, GaussianMixture, GaussianMixtureModel, BisectingKMeans, BisectingKMeansModel}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
80,application_1576053140655_0081,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


### 2)-Load Dataset

In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


### 3)-Data Preparation for Training

In [4]:
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_156af2ae1506
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_800222ba3fb7
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_138cab39cf8a
pipeline: org.apache.spark.ml.Pipeline = pipeline_ea877a034f01
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_ea877a034f01
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
val splitTime1 = data1.stat.approxQuantile("Time", Array(0.2), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132950.0
splitTime1: Double = 35926.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
trainingData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
val trainingData = trainingData0.unionAll(trainingData1)
val validationData = validationData0.unionAll(validationData1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [8]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|199029|
|  1.0|    98|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85286|
|  1.0|  394|
+-----+-----+



### 4)-Training the models

In [9]:
// Create a Kmeans Model with K=2
// train the model
val kmeans = new KMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val bisectingkmeans = new BisectingKMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val gaussianMixture = new GaussianMixture().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_431dc9ccfbe3
bisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeans = bisecting-kmeans_0eeb0a2fc8a0
gaussianMixture: org.apache.spark.ml.clustering.GaussianMixture = GaussianMixture_3686324677e5


In [10]:
// Fit that model to the training_data
val t = System.nanoTime
val modelKmeans = kmeans.fit(trainingData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359859917662803
modelKmeans: org.apache.spark.ml.clustering.KMeansModel = kmeans_431dc9ccfbe3
durationkmeans: Double = 15.555689667

initial model training finished.
Training process takes 15.555689667 secs


In [11]:
// Fit that model to the training_data
val t = System.nanoTime
val modelBisectingkmeans = bisectingkmeans.fit(trainingData)
val durationbisectingkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationbisectingkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359877306729827
modelBisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeansModel = bisecting-kmeans_0eeb0a2fc8a0
durationbisectingkmeans: Double = 36.668387885

initial model training finished.
Training process takes 36.668387885 secs


In [12]:
// Fit that model to the training_data
val t = System.nanoTime
val modelGMM = gaussianMixture.fit(trainingData)
val durationGMM = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationGMM secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359914891360605
modelGMM: org.apache.spark.ml.clustering.GaussianMixtureModel = GaussianMixture_3686324677e5
durationGMM: Double = 14.378401645

initial model training finished.
Training process takes 14.378401645 secs


### 5)- Prediction

In [13]:
 // Make predictions Kmeans
val t = System.nanoTime
var predictionsk = modelKmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsk = predictionsk.select(col("label"),col("clusters").cast("Double"),col("features"))
//val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)
predictionsk.groupBy("clusters").count().show()
//predictionsk.groupBy("label").count().show()
predictionsk.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359930285273247
predictionsk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.233953628

initial model training finished.
Training process takes 0.233953628 secs
predictionsk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|37242|
|     1.0|48438|
+--------+-----+

+-----+--------+--------------------+
|label|clusters|            features|
+-----+--------+--------------------+
|  0.0|     1.0|[0.83882606761928...|
|  0.0|     1.0|[0.82277302367350...|
|  0.0|     0.0|[0.85017862475755...|
|  0.0|     0.0|[0.86763723741192...|
|  0.0|     1.0|[0.83454159244389...|
|  0.0|     1.0|[0.81604628493588...|
|  0.0|     1.0|[0.82946538425151...|
|  0.0|     1.0|[0.86515125119032...|
|  0.0|     1.0|[0.81690960575875...|
|  0.0|     1.0|[0.82432595545760...|
+-----+--------+--------------------+
only showing top 10 rows



In [14]:
predictionsk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|36914|
|     1.0|  0.0|48372|
|     0.0|  1.0|  328|
|     1.0|  1.0|   66|
+--------+-----+-----+



In [15]:
 // Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsbk = modelBisectingkmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsbk = predictionsbk.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsbk.groupBy("clusters").count().show()
predictionsbk.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359935761131035
predictionsbk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.275609789

initial model training finished.
Training process takes 0.275609789 secs
predictionsbk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|48414|
|     1.0|37266|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85286|
|  1.0|  394|
+-----+-----+



In [16]:
predictionsbk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|48345|
|     1.0|  0.0|36941|
|     0.0|  1.0|   69|
|     1.0|  1.0|  325|
+--------+-----+-----+



In [17]:
// Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsgmm = modelGMM.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsgmm = predictionsgmm.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsgmm.groupBy("clusters").count().show()
predictionsgmm.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 359940252516158
predictionsgmm: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationkmeans: Double = 0.255112439

initial model training finished.
Training process takes 0.255112439 secs
predictionsgmm: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|61937|
|     1.0|23743|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85286|
|  1.0|  394|
+-----+-----+



In [18]:
predictionsgmm.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|61802|
|     1.0|  0.0|23484|
|     0.0|  1.0|  135|
|     1.0|  1.0|  259|
+--------+-----+-----+



### 6)-Evaluation

 ##### Calculation of Silhouette Score

In [19]:
// Evaluate clustering by computing Silhouette score
val evaluatorsi = new ClusteringEvaluator().setPredictionCol("clusters").setMetricName("silhouette")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorsi: org.apache.spark.ml.evaluation.ClusteringEvaluator = cluEval_0723496d0cec


In [20]:
val ScoreKmeans = evaluatorsi.evaluate(predictionsk)
println(s"Silhouette of Kmeans predictions with squared euclidean distance = $ScoreKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreKmeans: Double = 0.23770047920796275
Silhouette of Kmeans predictions with squared euclidean distance = 0.23770047920796275


In [21]:
val ScoreBisectingKmeans  = evaluatorsi.evaluate(predictionsbk)
println(s"Silhouette of Bisecting Kmeans predictions with squared euclidean distance = $ScoreBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreBisectingKmeans: Double = 0.23779764967687744
Silhouette of Bisecting Kmeans predictions with squared euclidean distance = 0.23779764967687744


In [22]:
val ScoreGMM  = evaluatorsi.evaluate(predictionsgmm)
println(s"Silhouette of Gaussian Mixture predictions with squared euclidean distance = $ScoreGMM")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreGMM: Double = 0.08114495355319234
Silhouette of Gaussian Mixture predictions with squared euclidean distance = 0.08114495355319234


In [23]:
// Evaluate clustering.
val costKmeans = modelKmeans.computeCost(trainingData)
println(s"Kmeans: Within Set Sum of Squared Errors = $costKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

costKmeans: Double = 3687.385796120448
Kmeans: Within Set Sum of Squared Errors = 3687.385796120448
costBisectingKmeans: Double = 3687.3818116274224
Bisecting Kmeans: Within Set Sum of Squared Errors = 3687.3818116274224
costBisectingKmeans: Double = 3687.3818116274224
Bisecting Kmeans: Within Set Sum of Squared Errors = 3687.3818116274224


In [24]:
val evaluatork1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatork2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatork3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatork4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracyk = evaluatork1.evaluate(predictionsk)
println("Area Under ROC Curve = " + areaUnderROCk.evaluate(predictionsk))
println("Accuracy = " + evaluatork1.evaluate(predictionsk))
println("Precision = " + evaluatork2.evaluate(predictionsk))
println("Recall = " + evaluatork3.evaluate(predictionsk))
println("F1 = " + evaluatork4.evaluate(predictionsk))
println("Test Error = " + (1.0 - accuracyk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatork1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ca12c11f3e03
evaluatork2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ea515f13e52e
evaluatork3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_21f260603038
evaluatork4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6c1ab466b90f
areaUnderROCk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_8b5803d519cc
accuracyk: Double = 0.4316059757236228
Area Under ROC Curve = 0.3001693555193388
Accuracy = 0.4316059757236228
Precision = 0.9866409993232748
Recall = 0.4316059757236228
F1 = 0.59978147500789
Test Error = 0.5683940242763772


In [25]:
val evaluatorbk1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorbk2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorbk3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorbk4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCbk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracybk = evaluatorbk1.evaluate(predictionsbk)
println("Area Under ROC Curve = " + areaUnderROCbk.evaluate(predictionsbk))
println("Accuracy = " + evaluatorbk1.evaluate(predictionsbk))
println("Precision = " + evaluatorbk2.evaluate(predictionsbk))
println("Recall = " + evaluatorbk3.evaluate(predictionsbk))
println("F1 = " + evaluatorbk4.evaluate(predictionsbk))
println("Test Error = " + (1.0 - accuracybk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorbk1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_574a99d8307a
evaluatorbk2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_de4b24b05a26
evaluatorbk3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_194cb0179618
evaluatorbk4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f8f449e91c08
areaUnderROCbk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_a50ba0d0b14c
accuracybk: Double = 0.5680438842203548
Area Under ROC Curve = 0.6958652469546778
Accuracy = 0.5680438842203548
Precision = 0.9940229441400761
Recall = 0.5680438842203548
F1 = 0.7199400303404286
Test Error = 0.4319561157796452


In [26]:
val evaluatorgmm1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorgmm2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorgmm3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorgmm4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCgmm = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracygmm = evaluatorgmm1.evaluate(predictionsgmm)
println("Area Under ROC Curve = " + areaUnderROCgmm.evaluate(predictionsgmm))
println("Accuracy = " + evaluatorgmm1.evaluate(predictionsgmm))
println("Precision = " + evaluatorgmm2.evaluate(predictionsgmm))
println("Recall = " + evaluatorgmm3.evaluate(predictionsgmm))
println("F1 = " + evaluatorgmm4.evaluate(predictionsgmm))
println("Test Error = " + (1.0 - accuracygmm))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorgmm1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_13694cc249e9
evaluatorgmm2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_757f4c5b7cdb
evaluatorgmm3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e30a77350f89
evaluatorgmm4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7c869e47c4f8
areaUnderROCgmm: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_ae9b06716ef0
accuracygmm: Double = 0.7243347338935574
Area Under ROC Curve = 0.691002272318485
Accuracy = 0.7243347338935574
Precision = 0.9932820455509879
Recall = 0.7243347338935574
F1 = 0.835807824606063
Test Error = 0.2756652661064426
